In [ ]:
from gnews import GNews
import pandas as pd
from datetime import datetime

gn = GNews(language='en', country='US', max_results=100)
gn.start_date = datetime(2025, 6, 1)
gn.end_date = datetime(2025, 8, 31)
news = gn.get_news('bitcoin (site:bitcoinmagazine.com)')

if not news:
    print("No se encontraron noticias de bitcoin.")
else:
    rows = [{"date": n.get("published date"), "title": n["title"], "source": n["publisher"]["title"], "url": n["url"]} for n in news]
    df = pd.DataFrame(rows).drop_duplicates(subset=["title"])

    df['date'] = pd.to_datetime(df['date'])

    df_filtered = df.groupby(df['date'].dt.date).head(1)

    df_filtered.to_csv("btc_bitcoinmagazine_gnews_filtered.csv", index=False)

In [3]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn.functional import softmax

input_file = "btc_bitcoinmagazine.csv"
df = pd.read_csv(input_file)

model_name = "MiguelZazu/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

prob_neg = []
prob_neu = []
prob_pos = []

for text in df['text']:
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=512
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = softmax(logits, dim=1).squeeze().tolist()

    prob_neg.append(probabilities[0])
    prob_neu.append(probabilities[1])
    prob_pos.append(probabilities[2])

df['prob_neg'] = prob_neg
df['prob_neu'] = prob_neu
df['prob_pos'] = prob_pos

output_file = "btc_bitcoinmagazine_analyze.csv"
df.to_csv(output_file, index=False, sep=";", float_format="%.4f")
print(f"Archivo guardado en: {output_file}")

Archivo guardado en: btc_bitcoinmagazine_analyze.csv


In [6]:
df_analyze = pd.read_csv(output_file, sep=";")
print(df_analyze.head())

         date                                              title  \
0  2025-08-30  Google’s Android Lockdown: Are You Really In C...   
1  2025-08-29                   Mark Stepheny - Bitcoin Magazine   
2  2025-08-28  Nunchuk Wallet Brings Programmable Bitcoin To ...   
3  2025-08-27      Lightning Is Misunderstood - Bitcoin Magazine   
4  2025-08-26  An Excerpt From Bitcoin Circular Economies: Th...   

             source                                                url  \
0  Bitcoin Magazine  https://news.google.com/rss/articles/CBMiowFBV...   
1  Bitcoin Magazine  https://news.google.com/rss/articles/CBMickFVX...   
2  Bitcoin Magazine  https://news.google.com/rss/articles/CBMiuAFBV...   
3  Bitcoin Magazine  https://news.google.com/rss/articles/CBMibEFVX...   
4  Bitcoin Magazine  https://news.google.com/rss/articles/CBMinwFBV...   

                                                text  prob_neg  prob_neu  \
0  Android, Google’s mobile operating system, ann...       0.0    1.00

In [ ]:
import pandas as pd
import shutil
import os

drive_path = "G:\\.shortcut-targets-by-id\\1-Fm633e95mI3RSHDX47L3xy-K9ubjynv\\Proyecto Final\\output\\sentimiento_diario_2025_actualizado.csv"
copy_drive_path = "G:\\.shortcut-targets-by-id\\1-Fm633e95mI3RSHDX47L3xy-K9ubjynv\\Proyecto Final\\output\\sentimiento_diario_2025_completo.csv"
local_path = "C:\\Users\\migue\\Documents\\Keep_Coding\\Neurotrader\\notebooks\\btc_bitcoinmagazine_analyze.csv"


if os.path.exists(drive_path):
    shutil.copy(drive_path, copy_drive_path)
    print("Copia creada.")
else:
    print("No se encontró el archivo de Drive.")
    exit()

df_drive = pd.read_csv(copy_drive_path, sep=";")
df_local = pd.read_csv(local_path, sep=";")

common_columns = list(set(df_drive.columns) & set(df_local.columns))
df_drive = df_drive[common_columns]
df_local = df_local[common_columns]

df_final = pd.concat([df_drive, df_local], ignore_index=True)

df_final.to_csv(copy_drive_path, index=False, sep=";")
print("Archivo final creado.")


Copia creada.
Archivo final creado.


In [10]:
import pandas as pd

dffff = pd.read_csv("sentimiento_diario_2025_completo_2.csv", sep=";")

dffff.tail(50)

,prob_neu,prob_neg,prob_pos,date
205,1.0,0.0,0.0,2025-08-20
206,1.0,0.0,0.0,2025-08-19
207,1.0,0.0,0.0,2025-08-18
208,0.4631,0.0001,0.5368,2025-08-15
209,0.9999,0.0001,0.0,2025-08-13
210,0.8135,0.0,0.1865,2025-08-12
211,1.0,0.0,0.0,2025-08-11
212,1.0,0.0,0.0,2025-08-09
213,1.0,0.0,0.0,2025-08-08
214,1.0,0.0,0.0,2025-08-07
